In [ ]:
%reset_selective -f a 

import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
from pathlib import Path
import numpy as np
TOP_N = 20

def df_to_json(df_to_save,f_name, title, tooltip):
    create_js = {
        'title': title.capitalize(), 
        'tooltip': tooltip.capitalize(),
        **json.loads(df_to_save.nlargest(TOP_N).to_json(orient="table"))
        }
    json.dump(create_js,open(JSON_OUTPUT.joinpath('{}.json'.format( f_name)), 'w', encoding='utf-8'), indent=4)

def multi_df_to_json(df_to_save,f_name, title, tooltip):
    create_js = {
        'title': title.capitalize(), 
        'tooltip': tooltip.capitalize(), 
        **json.loads(df_to_save.to_json(orient="table"))
        }
    json.dump(create_js,open(JSON_OUTPUT.joinpath('{}.json'.format(f_name)), 'w', encoding='utf-8'), indent=4)

for ELO in ["BRONZE", "DIAMOND", "GOLD", "IRON", "PLATINUM", "SILVER"]:

    JSON_OUTPUT = Path('json', ELO)
    JSON_OUTPUT.mkdir(exist_ok=True, parents=True)

    df = pd.read_csv(Path('dataset', ELO+'.csv'))
    # print(ELO, df.shape[0])
    del df['Unnamed: 0']

    for col in df.columns:
        df[col] = df[col].values.astype('int32')




    df_to_json(df['redTeamWin'].value_counts(), 'redWin','Red Win Rate', 'Win rate')

    # %%
    blue_bans = df['BlueBan1'].value_counts() + df['BlueBan2'].value_counts() + df['BlueBan3'].value_counts() + df['BlueBan4'].value_counts() + df['BlueBan5'].value_counts()
    df_to_json(blue_bans.nlargest(TOP_N), 'blueBans', 'Blue Team Bans', 'Number of bans')

    red_bans = df['RedBan1'].value_counts() + df['RedBan2'].value_counts() + df['RedBan3'].value_counts() + df['RedBan4'].value_counts() + df['RedBan5'].value_counts()
    df_to_json(red_bans, 'redBans', 'Mosted banned champions', 'Number of bans')

    # %%
    for role, full_name in [('Adc', 'Bot Lane'),('Sup', 'Support') ,('Mid', 'Middle Lane'), ('Jg', 'Jungle'), ('Top', 'Top Lane')]:
        tmp = (df['Blue'+role].value_counts()+ df['Red'+role].value_counts()).nlargest(TOP_N)
        df_to_json(tmp, 'mostPopular'+role, f'Most Popular {full_name} Champions', 'Most popular champion '+full_name)

    # %%
    # team that has banned nothing
    #  | df['BlueBan1'].isin([-1]) | df['BlueBan2'].isin([-1]) | df['BlueBan3'].isin([-1]) | df['BlueBan4'].isin([-1]) | df['BlueBan5'].isin([-1])
    df_no_ban = df[(df['RedBan1'].isin([-1]) | df['RedBan2'].isin([-1]) | df['RedBan3'].isin([-1]) | df['RedBan4'].isin([-1])  | df['RedBan5'].isin([-1]))]
    # team that has banned something
    df_ban = df[~(df['RedBan1'].isin([-1]) | df['RedBan2'].isin([-1]) | df['RedBan3'].isin([-1]) | df['RedBan4'].isin([-1])  | df['RedBan5'].isin([-1]))]

    # print(df_no_ban['redTeamWin'].value_counts(), "\n")
    df_to_json(df_no_ban['redTeamWin'].value_counts(), 'redWinNobans', 'Red Team Winrate without banning a champion', 'Win rate')
    # print(df_ban['redTeamWin'].value_counts(), "\n")
    df_to_json(df_ban['redTeamWin'].value_counts(), 'redWinBans', 'Red Team Winrate by at least banning a champion', 'Win rate')


    df_both_no_ban = df[(df['RedBan1'].isin([-1]) | df['RedBan2'].isin([-1]) | df['RedBan3'].isin([-1]) | df['RedBan4'].isin([-1])  | df['RedBan5'].isin([-1])| df['BlueBan1'].isin([-1]) | df['BlueBan2'].isin([-1]) | df['BlueBan3'].isin([-1]) | df['BlueBan4'].isin([-1]) | df['BlueBan5'].isin([-1]))]
    df_both_ban = df[~(df['RedBan1'].isin([-1]) | df['RedBan2'].isin([-1]) | df['RedBan3'].isin([-1]) | df['RedBan4'].isin([-1])  | df['RedBan5'].isin([-1])| df['BlueBan1'].isin([-1]) | df['BlueBan2'].isin([-1]) | df['BlueBan3'].isin([-1]) | df['BlueBan4'].isin([-1]) | df['BlueBan5'].isin([-1]))]

    # print(df_both_no_ban['redTeamWin'].value_counts(), "\n")
    # print(df_both_ban['redTeamWin'].value_counts())

    yasuo=157
    # https://leagueoflegends.fandom.com/wiki/Category:Knockup_champion
    with_knockups = ['266', '12', '53', '201', '31','105', '3','427', '40','59', '429', '141', '64', '117', '54', '267', '111', '20', '516', '78', '497', '421', '526', '92', '360', '113', '14', '517', '163', '161','264','5','777','154','143']
    with_knockups = set(map(int, with_knockups))
    redRoles = ['RedTop', 'RedJg', 'RedMid', 'RedAdc', 'RedSup']
    blueRoles = ['BlueTop', 'BlueJg','BlueMid', 'BlueAdc', 'BlueSup', 'redTeamWin']

    all_roles = set(redRoles+blueRoles)

    s = ""
    for role in all_roles:
        s+= "(df['{}']==157)|".format(role)

    yas_only_games = df[(df['RedTop']==157)|(df['RedJg']==157)|(df['RedMid']==157)|(df['RedAdc']==157)|(df['RedSup']==157)|(df['BlueTop']==157)|(df['BlueJg']==157)|(df['BlueMid']==157)|(df['BlueAdc']==157)|(df['BlueSup']==157)]
    yas_only_games = yas_only_games[all_roles]
    for c in yas_only_games.columns:
        if c == 'redTeamWin':
            continue
        yas_only_games[c] = yas_only_games[c].apply(lambda x: True if x in with_knockups else False)

    # convert bool to ints
    yas_only_games = yas_only_games*1
    yas_only_games = yas_only_games.astype('int32')
    yas_only_games['total_knockups'] = yas_only_games.drop('redTeamWin', axis=1).sum(axis=1)

    yas3_ban = yas_only_games[yas_only_games['total_knockups']>=3]['redTeamWin'].value_counts()
    yas2_ban = yas_only_games[yas_only_games['total_knockups']==2]['redTeamWin'].value_counts()
    yas1_ban = yas_only_games[yas_only_games['total_knockups']==1]['redTeamWin'].value_counts()
    yas0_ban = yas_only_games[yas_only_games['total_knockups']==0]['redTeamWin'].value_counts()


    yas_df = pd.DataFrame({
        "no knockup":yas0_ban,
        "one knockup": yas1_ban,
        "two knockup": yas2_ban,
        "three knockup or more": yas3_ban
    })
    multi_df_to_json(yas_df[1:].iloc[0], "yasuoGames", "Yasuo Wins per Number of Knockups in the Team", 'Number of knockups')


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
from pathlib import Path
from os.path import basename
import json

cred = credentials.Certificate("SA.json")
firebase_admin.initialize_app(cred)
db = firestore.client() 


for folder in ["BRONZE", "DIAMOND", "GOLD", "IRON", "PLATINUM", "SILVER"]:
    files = Path('json').joinpath(folder).glob("*.json")
    collection_name = folder.capitalize()
    tmp = []
    for file in files:
        with open(file) as fp:
            loaded = json.load(fp)
            tmp.append({file.stem: loaded })
            db.collection('statistics').document(collection_name).update({file.stem: loaded})

    # db.collection('statistics').document(collection_name).set([*tmp])
    # break

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=939b35d5-0867-4fd3-83c2-58e7fdcd481f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>